# Calcium event analysis

In this notebook, we will analyse the calcium imaging data that was acquired in the laboratory of Flavie Lavoie-Cardinal.

The data consists of a series of images that were acquired at a frame rate of 10 Hz. The raw data is not available into this directory. The entire dataset is available for download at this [link](https://s3.valeria.science/flclab-calcium/index.html). A subset of example videos can also be download from this [link]("https://s3.valeria.science/flclab-calcium/data/subset-testing-dataset.zip"). 

The data that is provided within this repository consists of the following:

An expert has annotated the data and identified the time points at which calcium events occur in the movies. The expert manually generated the annotation file (csv). The expert then extracted entire calcium traces at each (x, y) location in the movie and stored them in a `numpy` file. 

The data is stored into two folders:
- `data/annotations` contains the csv files
- `data/traces` contains the calcium traces for each detected events

The goal of this notebook is to analyse the data and to identify the calcium events automatically.

In [ ]:
import numpy
import os 
import glob
import pandas
import random
import scipy.sparse
import scipy.sparse.linalg

from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler

PATH = os.path.join("..", "data")
FREQ = 10 # Hz


# Load data

Implement a function named `load_traces` that loads the data from the given files. The function should take as input the path to the traces file and the path to the annotation file and return the following:
- `traces`: a numpy array of shape `(n, t)` where `n` is the number of traces and `t` is the number of time points
- `annotations`: a list of length `n` where `n` is the number of traces. Each element of the list is a list of time points at which the calcium events occur.

Verify that the function works by loading the data and plotting a random trace with the corresponding annotations.

In [111]:
def load_traces():
    pass

# Multiple traces on a single plot

Plot multiple traces on a single plot using matplotlib. We wish to reproduce a plot that will look similar to the plot below. You do not need to add the annotations on the plot.

![image](./img/traces.png)

# Baseline correction

Implement a function named `baseline_correction` that takes as input a trace and returns the baseline. It is recommended to do a baseline correction since the calcium traces are noisy and the baseline is not always at zero. Artifacts can be introduced in the acquisition process and the baseline can be shifted. For instance, the baseline can be shifted due to the movement of the sample or the movement of the microscope objective or ambient light. Different methods can be used to correct the baseline.

In the next exercice, the baseline correction should be done by implementing a polynomial fit of the trace. The degree of the polynomial fit should be a parameter of the function. The function should return the baseline.

Verify your implementation by plotting the trace and the baseline on the same plot.

In [113]:
def baseline_correction():
    pass


In the next exercice, we will use the already implemented baseline correction method to correct the baseline of all the traces. The method consists in the asymetrically reweighted penalized least squares smoothing (arPLS).

The arPLS method has two input parameters:
- `lam`: the regularization parameter
- `ratio` : the exit condition

Can you visualize the baseline corrected traces? What is the impact of the parameters on the baseline correction? What should be the optimal values for these parameters?

In [114]:
def baseline(y, lam=1e3, ratio=1e-6):
    """
    Baseline estimation using asymmetrically reweighted penalized least squares smoothing (arPLS) algorithm.

    :param y: input signal
    :param lam: regularization parameter
    :param ratio: exit condition

    :return: baseline

    :references:
    [1] Baek, S. J., Park, A., Ahn, Y. J., & Choo, J. (2015). Baseline correction using 
        asymmetrically reweighted penalized least squares smoothing. Analyst, 140(1), 250-257.
    """
    lam=1e5
    N = len(y)
    D = scipy.sparse.csc_matrix(numpy.diff(numpy.eye(N), 2))
    w = numpy.ones(N)
    MAX_ITER = 100

    for _ in range(MAX_ITER):
        W = scipy.sparse.spdiags(w, 0, N, N)
        Z = W + lam * D.dot(D.transpose())
        z = scipy.sparse.linalg.spsolve(Z, w * y)
        d = y - z
        # make d- and get w^t with m and s
        dn = d[d < 0]
        m = numpy.mean(dn)
        s = numpy.std(dn)
        wt = 1.0 / (1 + numpy.exp(2 * (d - (2 * s - m)) / s))
        # check exit condition and backup
        if numpy.linalg.norm(w - wt) / numpy.linalg.norm(w) < ratio:
            break
        w = wt

    return z


# DeltaF/F0 calculation

The variation of the fluorescence signal is often calculated as the relative change in fluorescence intensity compared to the baseline fluorescence intensity. This is often referred to as the deltaF/F0 calculation. Implement a function named `deltaF` that takes as input a trace, calculates the baseline using the `baseline` method implemented above and returns the deltaF/F0 trace.

In [ ]:
def deltaF():
    pass

# Signal preprocessing - Filters

One of the most important steps in the analysis of calcium imaging data is the preprocessing of the signal. The signal is often noisy and the calcium events are not always clearly visible. The signal can be preprocessed using different filters. The filters can be used to remove the noise from the signal, to smooth the signal or to enhance the signal.

In this exercise, we will implement a smoothing function to filter out the noise from the data. The following filters can be used:
- Median filter
- Mean filter
- Gaussian filter
- Savitzky-Golay filter

Implement a function named `filter_signal` that will apply the one of the filter to the trace. The function should take as input the signal and the name of filter to apply. The function should return the filtered signal. Optionally, you can add the parameters of the filter as input parameters of the function using `kwargs`.

Verify your implementation by applying the filter to the trace and plotting the original trace and the filtered trace on the same plot.

In [ ]:
def filter_signal():
    pass


# Signal preprocessing - Averaging

Another method that can be used to improve the signal-to-noise ratio is to average the signal over multiple trials or similar ROIs. The reason why averaging can improve the signal-to-noise ratio is because the noise is random and the signal is constant. The noise will cancel out when averaging the signal.

For more information, follow the discussion at this [link](https://chem.libretexts.org/Bookshelves/Analytical_Chemistry/Chemometrics_Using_R_(Harvey)/10%3A_Cleaning_Up_Data/10.2%3A_Signal_Averaging).

Implement a function named `average_signal` that will average the signal over multiple traces. The function should take as input the traces and corresponding positions and returne the averaged signal. 

Verify your implementation by averaging the signal over multiple traces and plotting the original traces and the averaged signal on the same plot.

*Hint.* The positions of the traces are stored in the annotation file. You can use the positions to average the signal. Consider that two traces can be grouped if they are close to each other. The distance between two traces can be calculated using the Euclidean distance. A distance of 5 pixels can be used as a threshold to group the traces.

In [116]:
def average_signal():
    pass


Using the `average_signal` method from above, implement a function named `load_average_traces` that returns the calcium traces and the annotations for the averaged traces both as lists.

In [115]:
def load_average_traces():
    pass


# Single event extraction

## Averaging

The calcium events are often extracted from the signal by averaging the signal over multiple events. This allows to measure the average shape of the calcium event. The average shape of the calcium event can be used to detect the calcium events in the signal.

In this exercise, we will extract the calcium events from the signal and plot them on a single plot. The calcium events can be extracted by taking the signal around the time points of the calcium events (+/- 1s). The signal can be averaged over multiple events to obtain the average shape of the calcium event.

Plot all calcium events from the dataset on a single plot using the provided annotations. Store all the calcium events in a list called `events`.

Using the `events` list, plot the average shape of the calcium event and the standard deviation of the calcium event.

## Modeling

Once the average shape of the calcium event is obtained, one can try to model the calcium event using quantitative parameters. In your opinion, what are the best parameters to model the calcium event?

From the averaged calcium event can you calculate the following parameters:
- Amplitude (-)
- Rise time (s)
- Decay time (s)

We will define the amplitude as the maximum value of the calcium event. We will define the rise time as the time it takes for the calcium event to go from 10% to 90%. The decay time is defined as the time it takes for the calcium event to decay from 90% to 10%.

The decay time constant can be calculated using the exponential decay model. The exponential decay model is defined as:
$$
y(t) = A \exp(- \lambda t) + C,
$$
where $A$ is the amplitude, $\lambda$ is the decay time constant and $C$ is the baseline.

Implement a curve fitting method to fit the averaged calcium event using the exponential decay model. Calculate the decay time constant of the averaged event using the exponential decay model.

Verify your implementation by plotting the averaged calcium event and the fitted model on the same plot.

*Hint.* You can use the `scipy.optimize.curve_fit` method to fit the model to the data.

In [ ]:
def exponential():
    pass


# Automatic event detection

The calcium events can be detected by thresholding the signal. In other words, an event will be considered when the signal is above a certain value. This threshold can be set to a fixed value for all calcium traces or can be set to a dynamic value. In the next exercise, we will try to detect the signals using both methods.

## Using a fixed threshold

What should be the threshold to detect all events that were detected by the expert?

Verify the threshold by plotting the calcium trace, the detected events, and the calculated threshold on the same plot.

*Hint.* Iterate over all calcium traces and calculate the signal that would detect all events in the trace.

*Hint.* The `scipy.signal.find_peaks` method can be used to detect the peaks in the signal.

## Using an adaptive threshold

In practice, events will not always be provided by the expert. Hence, it is important to detect the events automatically. As shown above using a fixed threshold can lead to errors in the detection of events. Typically, the threshold can be set to a dynamic value based on the noise level in the signal. The threshold can be set to a multiple of the noise level in the signal. This allows to adapt the detection threshold to the noise level in the signal.

What should be the threshold to detect all (or most) events based on the local noise?

Calculate the threshold for each trace based on the noise level in the trace. We will define the noise level as the standard deviation of the signal. The threshold can be set to a multiple of the noise level. Verify the threshold by plotting the calcium trace, the detected events, and the calculated threshold on the same plot.

Can you improve the threshold calculation by using a global noise level for each calcium traces?

*Advanced*

Implement a sliding window method to calculate the noise level in the signal. The noise level can be calculated as the standard deviation of the signal in the window.

Are all events real events? Can you filter out the false events using prior information about the shape of the events?

# Event detection efficiency

Even if the calcium events are detected automatically, it is important to evaluate the efficiency of the threshold that was used to detect the events. The efficiency of the threshold can be calculated by comparing the detected events with the expert annotations. In this section we will calculate the efficiency of the detection algorithm that we have implemented above.

The first step is to calculate wheter an event was detected or not. An event is considered detected if the detected event is within a certain time window of the expert annotation. The time window can be set to 1s.

From the association it is possible to calculate the following metrics:
1. Number of true positives
1. Number of false positives
1. Number of true negatives
1. Accuracy
1. Precision
1. Recall

*Hint.* Have a look at the linear sum assignment problem and how it can be used to calculate the efficiency of the threshold. See this [link](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html)

# Other methods for event detection

There exists many methods for event detection in calcium imaging data. Methods like SICT, Suite2p, CaImAn, etc. have been developed for event detection. The methods can incorporate a physics-based model of the calcium imaging data to improve the event detection or can also rely on machine learning methods to detect the events. Here are some of the methods that can be used for event detection:

1. Giovannucci, A., Friedrich, J., Gunn, P., Kalfon, J., Brown, B. L., Koay, S. A., Taxidis, J., Najafi, F., Gauthier, J. L., Zhou, P., Khakh, B. S., Tank, D. W., Chklovskii, D. B., & Pnevmatikakis, E. A. (2019). CaImAn an open source tool for scalable calcium imaging data analysis. eLife, 8, e38173. https://doi.org/10.7554/eLife.38173
1. Mancini, R., van der Bijl, T., Bourgeois-Jaarsma, Q., Lasabuda, R., & Groffen, A. J. (2018). SICT: Automated detection and supervised inspection of fast Ca2+ transients. Scientific Reports, 8(1), 15523. https://doi.org/10.1038/s41598-018-33847-4
1. Pachitariu, M., Stringer, C., Dipoppa, M., Schröder, S., Rossi, L. F., Dalgleish, H., Carandini, M., & Harris, K. D. (2017). Suite2p: Beyond 10,000 neurons with standard two-photon microscopy (p. 061507). bioRxiv. https://doi.org/10.1101/061507
1. Pachitariu, M., Stringer, C., & Harris, K. D. (2018). Robustness of Spike Deconvolution for Neuronal Calcium Imaging. Journal of Neuroscience, 38(37), 7976–7985. https://doi.org/10.1523/JNEUROSCI.3339-17.2018
1. Pnevmatikakis, E. A., Soudry, D., Gao, Y., Machado, T. A., Merel, J., Pfau, D., Reardon, T., Mu, Y., Lacefield, C., Yang, W., Ahrens, M., Bruno, R., Jessell, T. M., Peterka, D. S., Yuste, R., & Paninski, L. (2016). Simultaneous Denoising, Deconvolution, and Demixing of Calcium Imaging Data. Neuron, 89(2), 285–299. https://doi.org/10.1016/j.neuron.2015.11.037
1. Theis, L., Berens, P., Froudarakis, E., Reimer, J., Román Rosón, M., Baden, T., Euler, T., Tolias, A. S., & Bethge, M. (2016). Benchmarking Spike Rate Inference in Population Calcium Imaging. Neuron, 90(3), 471–482. https://doi.org/10.1016/j.neuron.2016.04.014
1. Vogelstein, J. T., Packer, A. M., Machado, T. A., Sippy, T., Babadi, B., Yuste, R., & Paninski, L. (2010). Fast nonnegative deconvolution for spike train inference from population calcium imaging. Journal of Neurophysiology, 104(6), 3691–3704. https://doi.org/10.1152/jn.01073.2009
1. Wang, Y., DelRosso, N. V., Vaidyanathan, T. V., Cahill, M. K., Reitman, M. E., Pittolo, S., Mi, X., Yu, G., & Poskanzer, K. E. (2019). Accurate quantification of astrocyte and neurotransmitter fluorescence dynamics for single-cell and population-level physiology. Nature Neuroscience, 22(11), Article 11. https://doi.org/10.1038/s41593-019-0492-2

*Can you implement any of the above methods for event detection?*

Verify online if the authors have provided the code for the above methods. If the code is not available, can you implement the method from scratch?

## Spike deconvolution using suite2p

In this section we will use the suite2p package to perform spike deconvolution. This is a very powerful package that can be used for spike deconvolution. This is reference *3* in the above list.

Information about the definition of spike deconvolution [is given in the suite2p documentation](https://suite2p.readthedocs.io/en/latest/FAQ.html#deconvolution-means-what). 

The deconvolution algorithm is based on the OASIS algorithm. The algorithm is fully described in the following paper: 
1. Friedrich, J., Zhou, P., & Paninski, L. (2017). Fast online deconvolution of calcium imaging data. PLoS computational biology, 13(3), e1005423.

How does the spike deconvolution algorithm performs compared to the thresholding method that we have implemented above?


In [ ]:
# Installs suite2p
!pip install "opencv-python-headless<4.3"
!pip install suite2p